In [23]:
from __future__ import print_function
import pandas as pd
import numpy as np

In [24]:
import re
dict_file = '../data/survey/7041-1996-2015_City_Survey_DataSF.xlsx'
data_dict = pd.read_excel(dict_file, 0, skiprows=3)
data_dict['Variable'] = data_dict['Variable'].str.replace('\n', '') 
data_dict = data_dict.set_index('Variable')

#keep only questions which have FDCBA grades
data_dict1 = data_dict.ix[['id', 'year', 'finweigh', 'zipcode', 'district', 'region']]
data_dict2 = data_dict[data_dict['Value Labels'].str.contains('F-Failing', flags=re.IGNORECASE).fillna(False)]
# remove column 'swcndnbd' as it's not present in data csv
data_dict2 = data_dict2.drop('swcndnbd')
data_dict = pd.concat([data_dict1, data_dict2])

In [25]:
survey_df = pd.read_csv('../data/survey/San_Francisco_City_Survey_Data_1996-2015.csv')

# keep only valid cols
survey_df = survey_df[list(set(data_dict.index) & set(survey_df.columns))]

# remove values of 6 and 7 (dont know, not provided responses)
replace_dict = {col: {6: np.nan, 7: np.nan} for col in data_dict2.index}
survey_df = survey_df.replace(replace_dict)

# consider data only from 2009 - 2015
survey_df = survey_df[survey_df['year'].isin([2009, 2011, 2013, 2015])]

# consider only 941XX zip codes for SF city
survey_df = survey_df[(survey_df['zipcode'] > 94100) & (survey_df['zipcode'] < 94200)]
print(survey_df.shape)

(10271, 60)


In [26]:
cat_qs_counts = data_dict2['Category'].value_counts()
cat_qs_counts

Recreation and Parks     14
Infrastructure           11
City Services General    11
Libraries                 9
Transportation            8
Children and Families     1
Name: Category, dtype: int64

In [27]:
## weight for each category is the count of the number of questions
weights = [cat_qs_counts[data_dict2.ix[col, 'Category']] for col in data_dict2.index]

#multiply responses by 'finweigh' to normalize across years
satisfaction_scores = (survey_df[data_dict2.index].multiply(survey_df['finweigh'], axis='rows') / weights).sum(axis=1) 
scores_by_zip = satisfaction_scores.groupby(survey_df['zipcode']).mean()

In [28]:
scores_by_zip.sort_values()

zipcode
94120.0     6.899325
94131.0     7.379361
94103.0     7.413292
94123.0     7.463880
94158.0     7.497538
94118.0     7.768823
94107.0     7.780656
94111.0     7.781247
94114.0     7.992968
94105.0     8.030017
94109.0     8.088264
94117.0     8.214848
94127.0     8.243457
94115.0     8.261622
94102.0     8.363183
94124.0     8.428608
94121.0     8.441895
94113.0     8.636639
94112.0     8.646508
94122.0     8.803248
94134.0     8.812299
94132.0     8.892504
94129.0     9.086159
94116.0     9.090505
94110.0     9.118152
94101.0     9.153678
94125.0     9.166556
94130.0     9.182060
94133.0     9.487025
94108.0     9.793031
94104.0     9.934013
94106.0    10.099214
94126.0    11.435526
94119.0    12.989787
dtype: float64

In [29]:
scores_by_zip.index = scores_by_zip.index.map(lambda x: int(x))
scores_by_zip

94101     9.153678
94102     8.363183
94103     7.413292
94104     9.934013
94105     8.030017
94106    10.099214
94107     7.780656
94108     9.793031
94109     8.088264
94110     9.118152
94111     7.781247
94112     8.646508
94113     8.636639
94114     7.992968
94115     8.261622
94116     9.090505
94117     8.214848
94118     7.768823
94119    12.989787
94120     6.899325
94121     8.441895
94122     8.803248
94123     7.463880
94124     8.428608
94125     9.166556
94126    11.435526
94127     8.243457
94129     9.086159
94130     9.182060
94131     7.379361
94132     8.892504
94133     9.487025
94134     8.812299
94158     7.497538
dtype: float64

In [31]:
# scores_by_zip.to_csv('/Users/chakri/bayeshack/analysis_data/scores_by_zipcodes.csv')